In [1]:
# =-=-=-=-=-=-=-=-=-=-=
# Data Load & Tokenize
# =-=-=-=-=-=-=-=-=-=-= 

import pandas
import re
from nltk.tokenize import WhitespaceTokenizer

# LOAD
colnames = ['author', 'title', 'date' , 'length', 'text']
df = pandas.read_csv('../data/talks_3.csv', names=colnames)
talks = df.text.tolist()
authors = df.author.tolist()
dates = df.date.tolist()
years = [re.sub('[A-Za-z ]', '', item) for item in dates]
authordate = [author+" "+year for author, year in zip(authors, years)]

# TOKENIZE
tokenizer = WhitespaceTokenizer()
texts = []
for talk in talks:   
    raw = re.sub(r"[^\w\d'\s]+",'', talk).lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(tokens)

In [2]:
# =-=-=-=-=-=-=-=-=-=-=
# Small Test Corpus
# =-=-=-=-=-=-=-=-=-=-= 

test = texts[0:5]

In [3]:
# =-=-=-=-=-=-=-=-=-=-=
# Function to collect word positions within a text (as a word list)
# =-=-=-=-=-=-=-=-=-=-= 

def word_positions(listname):
    from collections import defaultdict
    words_with_positions = defaultdict(list)
    for position, word in enumerate(listname):
        words_with_positions[word].append(float(position)/len(listname))
    return(words_with_positions)

# word_positions(texts[0]) # check output

In [54]:
# =-=-=-=-=-=-=-=-=-=-=
# Develop "super dictionary" of all the texts involved
# =-=-=-=-=-=-=-=-=-=-= 

super_dict = {}
for text in texts:
    temp_dict = word_positions(text)
    for k, v in temp_dict.items():
        if super_dict.get(k) is None:
            super_dict[k] = []
        if v not in super_dict.get(k):
            # Possibly problematic for larger data sets
            super_dict[k] = super_dict[k] + v
            
# print(super_dict) # check output

In [ ]:
super_dict["thank"]

In [63]:
# =-=-=-=-=-=-=-=-=-=-=
# Quick way to get occurrence and average position for a term in the dictionary
# =-=-=-=-=-=-=-=-=-=-= 

import statistics

word = "adaptable"

# the format method would work better here
print("*" + word + "* occurs " + str(len(super_dict[word])) + 
      " times in the corpus, with an average position of " + 
      str(statistics.mean(super_dict[word])) + ".")

*adaptable* occurs 21 times in the corpus, with an average position of 0.48408945020571637.


In [67]:
words_df = pandas.DataFrame()
words_df['word'] = super_dict.keys()
words_df['positions'] = super_dict.values()

# Check -----> returns 56588
# print(len(words_df))

56588

In [58]:
words_df.head()

,word,positions
0,casseroles,[0.7642857142857142]
1,procedurally,"[0.41314425961276247, 0.42568857376602126]"
2,gaelic,"[0.07612635939927499, 0.08130502330398758, 0.1..."
3,shambles,"[0.0924962852897474, 0.1502231036192365]"
4,gernot,[0.9419939577039275]


In [68]:
# I need two additional columns: 
# one for mean, one for counting the number of items in a list

positions = words_df['positions'].tolist()

average_positions = []
for the_list in positions:
    average_positions.append(sum(the_list)/len(the_list))
    
occurrences = []
for the_list in positions:
    occurrences.append(len(the_list))

# Quick Check
# print(average_positions[0:5])
# print(occurrences[0:5])

words_df['average'] = average_positions
words_df['occurs'] = occurrences
words_df.head(10)

,word,positions,average,occurs
0,casseroles,[0.7642857142857142],0.764286,1
1,procedurally,"[0.41314425961276247, 0.42568857376602126]",0.419416,2
2,gaelic,"[0.07612635939927499, 0.08130502330398758, 0.1...",0.098222,3
3,shambles,"[0.0924962852897474, 0.1502231036192365]",0.121360,2
4,gernot,[0.9419939577039275],0.941994,1
5,booker,"[0.6132478632478633, 0.6273504273504273]",0.620299,2
6,rewinds,[0.8224143664782175],0.822414,1
7,seditious,[0.9782886334610472],0.978289,1
8,traverse,"[0.34157998313185267, 0.34543761638733705, 0.7...",0.482537,7
9,sicilians,[0.9786196549137285],0.978620,1


In [69]:
words_df.to_csv('../data/word_places.csv', index=False)

In [ ]:
words_df.loc[df['occurs'] > 1000]